In [4]:
import numpy as np
import pandas as pd
import datetime as dt
import random
import yfinance as yf
import os
from os import listdir
from os.path import isfile, join

PATH = 'E:\\Projects\\Finoobs\\Portfolio Optimization\\stocks\\'
risk_free_rate = 0.0125

def get_df_from_csv(ticker):
    try:
        df = pd.read_csv(PATH + ticker + '.csv', index_col='Date', parse_dates=True)
    except FileNotFoundError:
        pass
    else:
        return df

def merge_df_by_column_name(col_name, sdate, edate, *tickers):
    mult_df = pd.DataFrame()
    sdate = pd.to_datetime(sdate)
    edate = pd.to_datetime(edate)

    for x in tickers:
        df = get_df_from_csv(x)
        df.index = pd.to_datetime(df.index)
        mask = (df.index >= sdate) & (df.index <= edate)
        mult_df[x] = df.loc[mask][col_name]
        
    return mult_df

def get_random_files(folder_path, num_files=None):
    file_list = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
    
    if num_files is None or num_files > len(file_list):
        num_files = len(file_list)
    
    random_files = random.sample(file_list, num_files)
    return [file[:-4] for file in random_files]

def optimize_portfolio(investment_amount):

    num_files_to_select = int(input("Enter the number of stocks: "))
    
    # port_list = get_random_files(PATH, num_files_to_select)
    port_list = [input(f"Enter input {i+1}: ") for i in range(num_files_to_select)]
    

    end_date = dt.datetime.now().strftime('%Y-%m-%d')
    mult_df = merge_df_by_column_name('Adj Close', '2019-01-01', end_date, *port_list)
    mult_df = mult_df.apply(pd.to_numeric, errors='coerce')

    returns = np.log(mult_df / mult_df.shift(1))
    
    p_ret = []
    p_vol = []
    p_SR = []
    p_wt = []

    port_list = [ticker + ".NS" for ticker in port_list]
    # print(port_list)
    
    for _ in range(10000):
        weights = np.random.random(num_files_to_select)
        weights /= np.sum(weights)
        
        ret = np.sum(weights * returns.mean()) * 252
        vol = np.sqrt(np.dot(weights.T, np.dot(returns.cov() * 252, weights)))
        sr = (ret - risk_free_rate) / vol
        
        p_ret.append(ret)
        p_vol.append(vol)
        p_SR.append(sr)
        p_wt.append(weights)

    p_ret = np.array(p_ret)
    p_vol = np.array(p_vol)
    p_SR = np.array(p_SR)
    p_wt = np.array(p_wt)

    optimal_idx = np.argmax(p_SR)
    optimal_weights = p_wt[optimal_idx]

    # Get current prices
    current_prices = {}
    for stock in port_list:
        # stock = stock+'.NS'
        ticker = yf.Ticker(stock)
        try:
            current_prices[stock] = ticker.info.get('currentPrice', np.nan)
            
        except KeyError:
            print(f"Price data for {stock} is not available.")
            continue
    print(current_prices)

    # Calculate number of shares and investment amount for each stock
    total_investment = 0
    recommendations = []
    for i, stock in enumerate(port_list):
        
        amount = investment_amount * optimal_weights[i]
        shares = int(amount / current_prices[stock])
        actual_investment = shares * current_prices[stock]
        total_investment += actual_investment
        recommendations.append({
            'stock': stock,
            'weight': optimal_weights[i],
            'shares': shares,
            'investment': actual_investment
        })

    return recommendations, p_ret[optimal_idx], p_vol[optimal_idx], total_investment

def main():
    investment_amount = float(input("Enter the total investment amount: "))
    recommendations, expected_return, volatility, total_invested = optimize_portfolio(investment_amount)

    print("\nInvestment Recommendations:")
    for rec in recommendations:
        print(f"Stock: {rec['stock']}")
        print(f"  Weight: {rec['weight']:.2%}")
        print(f"  Shares: {rec['shares']}")
        print(f"  Investment: ${rec['investment']:.2f}")
        print()

    print(f"Total Invested: ${total_invested:.2f}")
    print(f"Expected Annual Return: {expected_return:.2%}")
    print(f"Expected Volatility: {volatility:.2%}")
    print(f"Sharpe Ratio: {(expected_return - risk_free_rate) / volatility:.2f}")

if __name__ == "__main__":
    main()

AttributeError: 'NoneType' object has no attribute 'index'

In [2]:
import requests
import pandas as pd
import io
import time
import os
from splinter import Browser
from selenium import webdriver

# NSE Data Fetch
nse_url = "https://nsearchives.nseindia.com/content/equities/EQUITY_L.csv"
headers = {'User-Agent': 'Mozilla/5.0'}

# Create Session for NSE data fetch
with requests.Session() as s:
    s.headers.update(headers)
    response = s.get(nse_url)
    df_nse = pd.read_csv(io.BytesIO(response.content))

# BSE Data Fetch
download_dir = "E:\\Projects\\Finoobs\\Portfolio Optimization\\tmp"
bse_link = "https://mock.bseindia.com/corporates/List_Scrips.html"

# Set download preferences for headless browsing
prefs = {"download.default_directory": download_dir}
options = webdriver.ChromeOptions()
options.add_experimental_option("prefs", prefs)
options.add_argument("--headless")  # Run headless
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

# Initialize browser and download BSE data
try:
    browser = Browser('chrome', options=options)
    browser.visit(bse_link)
    
    # Fill form and submit for BSE data download
    browser.find_by_id('ddlsegment').select("Equity")
    browser.find_by_id('ddlstatus').select("Active")
    browser.find_by_id('btnSubmit').click()
    browser.is_element_present_by_text("Issuer Name", wait_time=10)
    browser.find_by_id('lnkDownload').click()
    
    # Wait for download to complete
    file_path = os.path.join(download_dir, "Equity.csv")
    wait_time = 0
    while not os.path.exists(file_path) and wait_time < 30:
        time.sleep(1)
        wait_time += 1
    
    if os.path.exists(file_path):
        df_bse = pd.read_csv(file_path)
    else:
        print("Download failed or file not found.")
        df_bse = pd.DataFrame()
        
finally:
    browser.quit()

# Data Cleaning and Merging
if not df_bse.empty:
    df_bse = df_bse.rename(columns={"Security Name": "NAME OF COMPANY", "Security Id": "SYMBOL"})
    df_bse['NAME OF COMPANY'] = df_bse['NAME OF COMPANY'].str.title()
    df_nse['NAME OF COMPANY'] = df_nse['NAME OF COMPANY'].str.title()
    
    # Select relevant columns
    df_bse = df_bse[['SYMBOL', 'NAME OF COMPANY']]
    df_nse = df_nse[['SYMBOL', 'NAME OF COMPANY']]
    
    # Merge on SYMBOL and prioritize NSE company names if available
    final_df = pd.merge(df_nse, df_bse, on='SYMBOL', how="outer", suffixes=('_NSE', '_BSE'))
    final_df['NAME OF COMPANY'] = final_df['NAME OF COMPANY_NSE'].combine_first(final_df['NAME OF COMPANY_BSE'])
    final_df = final_df[['SYMBOL', 'NAME OF COMPANY']]
    
    # Display or process final merged DataFrame
    print(final_df)
else:
    print("No BSE data to merge.")


         SYMBOL                                    NAME OF COMPANY
0         08ABB  Nippon India Equity Savings Fund -  Segregated...
1         08ADD  Nippon India Equity Savings Fund -  Segregated...
2         08ADR  Nippon India Equity Savings Fund -  Segregated...
3         08AGG  Nippon India Equity Savings Fund -  Segregated...
4         08AMD  Nippon India Equity Savings Fund -  Segregated...
...         ...                                                ...
4529      ZUARI                       Zuari Agro Chemicals Limited
4530   ZUARIIND                           Zuari Industries Limited
4531   ZWELCAST                                Welcast Steels Ltd.
4532  ZYDUSLIFE                         Zydus Lifesciences Limited
4533  ZYDUSWELL                             Zydus Wellness Limited

[4534 rows x 2 columns]


In [5]:
final_df.to_csv('E:\Projects\Finoobs\Portfolio Optimization\Project\companies.csv',index=False)

In [5]:
import pandas as pd
df = pd.read_csv('E:\Projects\Finoobs\Portfolio Optimization\Project\companies.csv')
df['SYMBOL'] = df['SYMBOL'].apply(lambda x: x if x.endswith('.NS') else x + '.NS')
df.to_csv('E:\Projects\Finoobs\Portfolio Optimization\Project\companies.csv',index=False)